<a href="https://colab.research.google.com/github/zoeliving/next-platform-starter/blob/main/videoSegmentCutting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Install ffmpeg-python (Python wrapper for ffmpeg)
!pip install ffmpeg-python
!apt-get install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [25]:
# Mount Google Drive:
# Ensures that Colab can access files in your Google Drive.
from google.colab import files, drive
import os

# Step 1: Ask for output storage preference
output_location = input("Where would you like to store output? (Enter 'gdrive' for Google Drive or 'local' for your computer): ").strip().lower()

# Initialize output directory path
if output_location == "gdrive":
    # Mount Google Drive
    drive.mount('/content/drive')

    # Ask for project folder name in Google Drive
    project_name = input("Enter the project folder name for Google Drive storage: ").strip()
    output_dir = f"/content/drive/My Drive/{project_name}/video_segments/"
    os.makedirs(output_dir, exist_ok=True)
    print(f"Output will be stored in Google Drive at: {output_dir}")

elif output_location == "local":
    # Ask user to specify output directory on their computer
    output_dir = "/content/video_segments/"
    os.makedirs(output_dir, exist_ok=True)
    print(f"Output will be stored temporarily in Colab. You can download files from: {output_dir}")

else:
    print("Invalid choice. Please enter 'gdrive' or 'local'.")
    raise SystemExit


# Verify the directory
print("Contents of My Drive:")
!ls "/content/drive/My Drive/"


Where would you like to store output? (Enter 'gdrive' for Google Drive or 'local' for your computer): gdrive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter the project folder name for Google Drive storage: Dr Karen Kan & David Schmidt: Advanced Protocols
Output will be stored in Google Drive at: /content/drive/My Drive/Dr Karen Kan & David Schmidt: Advanced Protocols/video_segments/
Contents of My Drive:
'Colab Notebooks'			     'Dr Karen Kan & David Schmidt: Advanced Protocols'
'Dr Kan & David Schmidt Advanced Protocols'   video_clips


In [ ]:
# Step 2: Ask for input file source
input_location = input("Where are your input files located? (Enter 'gdrive' for Google Drive or 'local' to upload from your computer): ").strip().lower()

# Initialize input paths
if input_location == "gdrive":
    # Get video and CSV filenames from Google Drive
    video_filename = input("Enter the name of the video file in Google Drive (e.g., video.mp4): ").strip()
    csv_filename = input("Enter the name of the CSV file in Google Drive (e.g., segments.csv): ").strip()
    input_video_path = f"/content/drive/My Drive/{project_name}/{video_filename}"
    csv_path = f"/content/drive/My Drive/{project_name}/{csv_filename}"
    print(f"Using input files from Google Drive at:\n Video: {input_video_path}\n CSV: {csv_path}")

elif input_location == "local":
    # Upload video file
    print("Please upload the video file from your computer.")
    uploaded_video = files.upload()
    video_filename = list(uploaded_video.keys())[0]
    input_video_path = f"/content/{video_filename}"

    # Upload CSV file
    print("Please upload the CSV file from your computer.")
    uploaded_csv = files.upload()
    csv_filename = list(uploaded_csv.keys())[0]
    csv_path = f"/content/{csv_filename}"

    print(f"Using input files uploaded from your computer:\n Video: {input_video_path}\n CSV: {csv_path}")

else:
    print("Invalid choice. Please enter 'gdrive' or 'local'.")
    raise SystemExit

# Final confirmation
print(f"Project setup complete:\n Input Video Path: {input_video_path}\n CSV Path: {csv_path}\n Output Directory: {output_dir}")

In [ ]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import pandas as pd
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

# Define paths for input files
# Specify the path for the CSV and the video file
csv_path = "/content/drive/My Drive/segments.csv"  # Replace with your actual path
input_video_path = "/content/drive/My Drive/input_video.mp4"  # Replace with your actual path

# Flexible time parsing function
def time_to_seconds(time_str):
    try:
        # Try parsing as 'hh:mm:ss'
        time_obj = datetime.strptime(time_str, '%H:%M:%S')
        return time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second
    except ValueError:
        # Fallback to 'mm:ss' if no hours present
        time_obj = datetime.strptime(time_str, '%M:%S')
        return time_obj.minute * 60 + time_obj.second

# Read CSV file for segment details
segments_df = pd.read_csv(csv_path)


In [ ]:
""" Explanation of the Code Sharpening:

The sharpen_video function uses ffmpeg-python to apply an unsharp filter. Adjust the parameters:
luma_msize_x and luma_msize_y: Control the matrix size for sharpening.
luma_amount: Controls the sharpness intensity. Increase for more sharpening (e.g., 1.5 to 2).
Splitting:

The split_video function reads the CSV and uses the sharpened video as input for segment extraction.
Parameters in the unsharp Filter
luma_msize_x and luma_msize_y: Define the size of the blur kernel. Larger values
 apply more significant blurring, which is then subtracted to enhance edges.
luma_amount: Controls the intensity of sharpening. A value of 1.0 is neutral;
values above 1.0 increase sharpness.

Running the Code
Upload your files (segments.csv and input_video.mp4) to Google Drive.
Update csv_path, input_video_path, and other paths in the code as needed.
Run all cells in the notebook. The code will create a sharpened version of the input video, then save each segment from the sharpened video.

Notes
Performance: Sharpening can be resource-intensive, but the free Colab CPU runtime can handle moderate sharpening tasks.
Adjust Sharpening: Experiment with luma_amount to control sharpness. Lower values may help avoid over-sharpening artifacts.
This configuration should work well for enhancing and cutting your video in Google Colab.
"""


' Explanation of the Code Sharpening:\n\nThe sharpen_video function uses ffmpeg-python to apply an unsharp filter. Adjust the parameters:\nluma_msize_x and luma_msize_y: Control the matrix size for sharpening.\nluma_amount: Controls the sharpness intensity. Increase for more sharpening (e.g., 1.5 to 2).\nSplitting:\n\nThe split_video function reads the CSV and uses the sharpened video as input for segment extraction.\nParameters in the unsharp Filter\nluma_msize_x and luma_msize_y: Define the size of the blur kernel. Larger values\n apply more significant blurring, which is then subtracted to enhance edges.\nluma_amount: Controls the intensity of sharpening. A value of 1.0 is neutral; \nvalues above 1.0 increase sharpness.\n\nRunning the Code\nUpload your files (segments.csv and input_video.mp4) to Google Drive.\nUpdate csv_path, input_video_path, and other paths in the code as needed.\nRun all cells in the notebook. The code will create a sharpened version of the input video, then sav

In [ ]:
# Process each segment in the CSV file and output segment video file
def split_video(input_path, segments_df, output_dir):
    for _, row in segments_df.iterrows():
        segment_title = row['segment_title']
        start_time = row['starttime']
        end_time = row['endtime']

        # Generate output file name and path
        output_filename = f"{segment_title}.mp4"
        output_path = os.path.join(output_dir, output_filename)

        # Extract the segment
        ffmpeg_extract_subclip(input_path, start_time, end_time, targetname=output_path)
        print(f"Saved: {output_filename} from {start_time} to {end_time} seconds")



In [ ]:
# Run the split function without sharpening.
# skip if you want sharpening
split_video(input_video_path, segments_df, output_dir)

NameError: name 'split_video' is not defined

In [ ]:
# optional:  Sharpen the original video before cutting.  skip if unneeded.
# Function to sharpen video using ffmpeg-python
def sharpen_video(input_path, output_path):
    (
        ffmpeg
        .input(input_path)
        .filter('unsharp', luma_msize_x=5, luma_msize_y=5, luma_amount=2)
        .output(output_path)
        .run(overwrite_output=True)
    )
    print(f"Sharpened video saved to {output_path}")

In [ ]:
import ffmpeg
# Sharpened video output path
sharpened_video_path = "/content/drive/My Drive/sharpened_videos/sharpened_input_video.mp4"

# Create the directory if it doesn't exist
os.makedirs(sharpened_video_path, exist_ok=True)

# Step 1: Sharpen the video
sharpen_video(input_video_path, sharpened_video_path)

# Step 2: Load segments and split sharpened video
segments_df = pd.read_csv(csv_path)
split_video(sharpened_video_path, segments_df, output_dir)
